In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
import torch

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert_chatbot")
model = BertForSequenceClassification.from_pretrained("bert_chatbot")
label_encoder = joblib.load("label_encode.pkl")

In [11]:
# Intent prediction function with optimizations
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure same device
    
    with torch.no_grad():  # Disable gradient calculation for efficiency
        outputs = model(**inputs)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred_label = torch.argmax(probs, dim=1).item()
    intent = label_encoder.inverse_transform([pred_label])[0]
    return intent

In [12]:
predict_intent("Hello")

'hello there tell me how are you feeling today'

In [13]:
predict_intent("I am feeling bad")

'why do you think you feel this way'

In [16]:
predict_intent("give some advices, i am feeling not well")

'as exasperated as you feel and as obvious it is to you that your husband cares more about drinking than being honest with you changing this is up to him does he care that you re upset with him for drinking more than he can handle does he care about you more than drinking living with an alcoholic which is how his behavior sounds is lonely and frustrating concentrate on what you like about him as he is now with his drinking your happiness matters his ignoring of what you re talking about doesn t mean your requests are unreasonable be prepared to learn to live with and accept how he behaves or not avoid the expectation that you can convince him to change people change only when they realize their life can improve by changing he doesn t sound like this at least now'